In [1]:
import matplotlib as mpl
mpl.use("Agg")
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as manimation
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import numpy as np
from numpy import *
from qutip import *
from scipy.stats import norm

In [2]:
N = 50

wr = 10.0 * 2 * pi
wq1 = 9.0 * 2 * pi

g1 = 0.1 * 2 * pi

chi1 = g1**2 / abs(wr-wq1)

wd1 = 10.0 * 2 * pi

In [3]:
tlist = np.linspace(0, 0.20, 10000)

In [4]:
# cavity operators
a = tensor(destroy(N), qeye(2))
n = a.dag() * a

# operators for qubit 1
sm1 = tensor(qeye(N), destroy(2))
sz1 = tensor(qeye(N), sigmaz())
n1 = sm1.dag() * sm1

In [ ]:
Hsyst = wr*n + wq1*sz1/2 + g1*(a*sm1.dag() + a.dag()*sm1)

In [ ]:
Hsyst

In [ ]:
def gaussianpulse(x,ts,tf):
    s = (tf-ts)/4
    m = (ts+tf)/2
    return (np.heaviside(x-m+2*s,1)-np.heaviside(x-m-2*s,1))*norm.pdf(x, loc = m, scale = s)/0.9544997361

def step_t(w1, w2, t0, width, t):
    """
    Step function that goes from w1 to w2 at time t0
    as a function of t. 
    """
    return w1 + (w2 - w1) * (t > t0)


fig, axes = plt.subplots(1, 1, figsize=(8,2))
axes.plot(tlist, [step_t(0.5, 1.5, 5, 0.0, t) for t in tlist], 'k')
axes.set_ylim(0, 2)
fig.tight_layout()

Introducir un pluso de amplitud x wr y duración 0.05ns a la frecuencia de la cavidad desplaza la cavidad hacia -x en p

Un pulso de -j x wr después de n 2pi/wr regresa la cavidad al estado vacío

Dispersive shift: g^2/abs(wq-wr)

Publication: Optimised coherent controlization! No hace falta esperar 4Dt, sino Dt+porción de vuelta exacta A en función de Gate Time+porción de vuelta exacta B en función de Gate Time+Dt

In [ ]:
def ksi1_t(t, args=None):
    return wr/(2*np.pi) * gaussianpulse(t,0.075,0.125) *abs(1)

def ksi1_t1(t, args=None):
    return wr/(2*np.pi) * gaussianpulse(t,0.075,0.125) *(1) * exp(-1j*wd1*(t))

def ksi1_t2(t, args=None):
    return wr/(2*np.pi) * gaussianpulse(t,0.075,0.125) *(1) * exp(1j*wd1*(t))

def ksi2_t(t, args=None):
    return 2.5*wr*((step_t(0.0, 1, 200.30, 0.0, t)-step_t(0.0, 1, 200.35, 0.0, t))*abs(1))

def ksi2_t1(t, args=None):
    return 2.5*wr*((step_t(0.0, 1, 200.30, 0.0, t)-step_t(0.0, 1, 200.35, 0.0, t))*(1)) * exp(-1j*wd1*(t-200.30))

def ksi2_t2(t, args=None):
    return 2.5*wr*((step_t(0.0, 1, 200.30, 0.0, t)-step_t(0.0, 1, 200.35, 0.0, t))*(1)) * exp(1j*wd1*(t-200.30))

In [ ]:
psi0 = tensor(basis(N,0),(basis(2,0)+basis(2,1))/sqrt(2))
H_t = [[a, ksi1_t1], [a.dag(), ksi1_t2], [a, ksi2_t1], [a.dag(), ksi2_t2], Hsyst]

In [ ]:
res = mesolve(H_t, psi0, tlist, [], [])

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(12,8))

axes[0].plot(tlist, array(list(map(ksi1_t, tlist))) / (2*pi), 'b', linewidth=2, label="drive 1")
axes[0].plot(tlist, array(list(map(ksi2_t, tlist))) / (2*pi), 'r', linewidth=2, label="drive 2")
axes[0].set_ylim(0, 26)
axes[0].set_ylabel("Energy (GHz)", fontsize=16)
axes[0].legend()

axes[1].plot(tlist, real(expect(n, res.states)), 'r', linewidth=2, label="q")
#axes[1].plot(tlist, real(expect((a.dag() + a)/2, res.states)), 'r', linewidth=2, label="q")
#axes[1].plot(tlist, real(expect(1j*(a.dag() - a)/2, res.states)), 'g', linewidth=2, label="p")
axes[1].plot(tlist, real(expect(n1, res.states)), 'b', linewidth=2, label="qubit 1")
axes[1].set_ylim(0, 50)

axes[1].set_xlabel("Time (ns)", fontsize=16)
axes[1].set_ylabel("Occupation probability", fontsize=16)
axes[1].legend()

fig.tight_layout()

In [ ]:
# extract the final state from the result of the simulation
rho_final = res.states[49825]

In [ ]:
# trace out the resonator mode and print the two-qubit density matrix
rho_qubits = ptrace(rho_final, 1)
rho_qubits

In [ ]:
# compare to the ideal result of the sqrtiswap gate (plus phase correction) for the current initial state
rho_qubits_ideal = ket2dm(tensor(basis(2,1),basis(2,1)))
rho_qubits_ideal

In [ ]:
fidelity(rho_qubits, rho_qubits_ideal)

In [ ]:
concurrence(rho_qubits)

In [ ]:
wd3

In [ ]:
def plot_wigner_2d(psi):
    #fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(12, 6))
    fig = plt.figure(figsize=(8, 8))
    
    ax = fig.add_subplot(1, 1, 1)
    plot_wigner(psi, fig=fig, ax=ax, alpha_max=10);
    
    plt.close(fig)
    return fig

def plot_wigner_2d3(psi1, psi2, psi3):
    #fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(12, 6))
    fig = plt.figure(figsize=(15, 5))
    
    ax = fig.add_subplot(1, 3, 1)
    plot_wigner(psi1, fig=fig, ax=ax, alpha_max=10);
    
    ax = fig.add_subplot(1, 3, 2)
    plot_wigner(psi2, fig=fig, ax=ax, alpha_max=10);
    
    ax = fig.add_subplot(1, 3, 3)
    plot_wigner(psi3, fig=fig, ax=ax, alpha_max=10);
    
    
    plt.close(fig)
    return fig

In [ ]:
#psi1 = ptrace(res.states[49924],0) #una vuelta = 0.2ns = 2pi/wr
#psi2 = ptrace(res.states[49925],0) #una vuelta = 0.2ns = 2pi/wr
#psi3 = ptrace(res.states[49926],0) #una vuelta = 0.2ns = 2pi/wr
#plot_wigner_2d3(psi1,psi2,psi3)

psi = ptrace(res.states[-1],0) #una vuelta = 0.2ns = 2pi/wr
plot_wigner_2d(psi)

In [ ]:
FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib',
                comment='Movie support!')
writer = FFMpegWriter(fps=15, metadata=metadata)

fig = plt.figure(figsize=(5, 5))
    
ax = fig.add_subplot(1, 1, 1)

with writer.saving(fig, "writer_test5.mp4", 60):
    for i in range(137):
        plot_wigner(ptrace(res.states[49800+i],0), fig=fig, ax=ax, alpha_max=10);
        writer.grab_frame()

In [ ]:
2*pi/(g1**2/abs(wq1-wr))

In [11]:
N = 50

wr = 7.0 * 2 * np.pi
wq = np.array([5.0 * 2 * np.pi, 6.0 * 2 * np.pi, 8.0 * 2 * np.pi, 9.0 * 2 * np.pi])

g = np.array([0.1*np.sqrt(2) * 2*np.pi, 0.1 * 2*np.pi, 0.1 * 2*np.pi, 0.1*np.sqrt(2) * 2*np.pi])

D = wq - wr

chi = g**2 / abs(wr-wq)

In [14]:
2*pi/chi

array([ 100.,  100.,  100.,  100.])

In [10]:
0.1*np.sqrt(2)

0.14142135623730953